In [2]:
import os
import sys
import math
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pickle

# Menu
<a name="navegacao"></a>

## 1) [Preparação dos dados](#parte1)
- 1.1 [Leitura dos Dados de Exemplo](#exemplo)
- 1.2 [Leitura das Coordenadas](#coordenadas)
- 1.3 [Mapa do Exemplo](#mapaexemplo)
- 1.4 [Leitura dos Dados de Fraudes](#dadosfraudes)
- 1.5 [Mapa das Fraudes de PF](#mapafraudepf)
- 1.6 [Mapa das Fraudes de PJ](#mapafraudepj)



<a name="exemplo"></a>

## 1.1) Leitura dos Dados de Exemplo

In [3]:
# Importando bibliotecas necessárias
import pandas as pd
#Em seguida, é preciso Carregar o conjunto de dados no Python. Para tanto utilizamos o read_csv do pandas:

# Importando o conjunto de dados
covid = pd.read_csv(
    filepath_or_buffer = 'https://raw.githubusercontent.com/jonates/opendata/master/covid19_brasil/cases-brazil-cities-20220811.csv', 
    sep=',',
    decimal='.'
)

In [4]:
covid.tail()

,country,state,city,ibgeID,deaths,totalCases,deaths_per_100k_inhabitants,totalCases_per_100k_inhabitants,deaths_by_totalCases,_source,date,newCases,newDeaths,last_info_date
5591,Brazil,RS,Áurea/RS,4301552,7,598,199.03327,17003.12767,0.01171,MS,2022-08-11,0,0,2022-08-11
5592,Brazil,PR,Ângulo/PR,4101150,10,882,341.18048,30092.11873,0.01134,MS,2022-08-11,0,0,2022-08-11
5593,Brazil,BA,Érico Cardoso/BA,2900504,6,666,57.07220,6335.01379,0.00901,MS,2022-08-11,0,0,2022-08-11
5594,Brazil,PA,Óbidos/PA,1505106,163,8590,310.63595,16370.32379,0.01898,MS,2022-08-11,0,0,2022-08-11
5595,Brazil,SP,Óleo/SP,3533809,5,159,204.33183,6497.75235,0.03145,MS,2022-08-11,0,0,2022-08-11


<a name="coordenadas"></a>

## 1.2) Leitura das Coordenadas

In [5]:
import json

# Abra o arquivo JSON e carregue os dados
with open('geojs-100-mun.json', 'r', encoding='utf-8') as f:
    geo_cidades = json.load(f)

In [6]:
geo_cidades['features'][1]


{'type': 'Feature',
 'properties': {'id': '1100023',
  'name': 'Ariquemes',
  'description': 'Ariquemes'},
 'geometry': {'type': 'Polygon',
  'coordinates': [[[-62.5359497334, -9.7318235272],
    [-62.5078156831, -9.7542128151],
    [-62.5071949714, -9.7610327028],
    [-62.4931988248, -9.7722945792],
    [-62.4656434329, -9.7779520607],
    [-62.455475152, -9.7735643212],
    [-62.4487884003, -9.7798371306],
    [-62.4352854171, -9.780129528],
    [-62.427515507, -9.78922021],
    [-62.4220885261, -9.8042236458],
    [-62.4226463125, -9.8054333146],
    [-62.4381431726, -9.8183977831],
    [-62.4433117918, -9.8179401146],
    [-62.4470562726, -9.8217753739],
    [-62.4546209912, -9.8624535704],
    [-62.4492003622, -9.8928788296],
    [-62.452945144, -9.8981477887],
    [-62.4528404432, -9.9057639907],
    [-62.4628028052, -9.9154521956],
    [-62.4625555024, -9.9301223611],
    [-62.488131444, -9.9555415354],
    [-62.503700399, -9.9645808413],
    [-62.5089467211, -9.9949554099],
  

In [7]:
# Renomeando o ibgeID para id igual ao do arquivo geoespacial
covid = covid.rename(columns={'ibgeID':'id'})

# transformando o 'id' para tipo object, o mesmo do geoespacial
covid['id'] = covid['id'].astype('str')

# Espiando resultado da estrutura
covid.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5596 entries, 0 to 5595
Data columns (total 14 columns):
 #   Column                           Non-Null Count  Dtype  
---  ------                           --------------  -----  
 0   country                          5596 non-null   object 
 1   state                            5596 non-null   object 
 2   city                             5596 non-null   object 
 3   id                               5596 non-null   object 
 4   deaths                           5596 non-null   int64  
 5   totalCases                       5596 non-null   int64  
 6   deaths_per_100k_inhabitants      5596 non-null   float64
 7   totalCases_per_100k_inhabitants  5596 non-null   float64
 8   deaths_by_totalCases             5596 non-null   float64
 9   _source                          5596 non-null   object 
 10  date                             5596 non-null   object 
 11  newCases                         5596 non-null   int64  
 12  newDeaths           

<a name="mapaexemplo"></a>

## 1.3) Mapa do Exemplo


In [8]:
# Importando biblioteca necessaria
import plotly.express as px


In [12]:
# Criando o mapa
mapa_mortalidade_covid = px.choropleth_mapbox(
    data_frame = covid, 
    geojson = geo_cidades, 
    locations='id', 
    featureidkey='properties.id',
    color='deaths_per_100k_inhabitants',
    color_continuous_scale= 'reds',
    range_color=(150, 450),
    mapbox_style='open-street-map',
    zoom=2.5, 
    center = {"lat": -17.14, "lon": -57.33},
    opacity=1,
    labels={'deaths_per_100k_inhabitants':'Tx. Mortalidade (/ 100 mil hab.)', 
            'id' : 'Código do município'
    },
    width = 1000,
    height = 800,
    title = 'Taxa de Mortalidade por Covid-19, por municípios, 11/08/2022'
)

# Ajustando as margens
#mapa_mortalidade_covid.update_layout(margin={'r':0,'t':0,'l':0, 'b':0})

# Reduzindo a largura das bordas dos municípios
#mapa_mortalidade_covid.update_traces(marker_line_width=0.01)

# Exibindo o mapa
#mapa_mortalidade_covid.show()


<a name="dadosfraudes"></a>

## 1.4) Leitura dos Dados de Fraudes

<a name="mapafraudepf"></a>

## 1.5) Mapa das Fraudes de PF


<a name="mapafraudepj"></a>

## 1.6) Mapa das Fraudes de PJ